In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [49]:
from k12libs.utils.nb_easy import k12ai_start_html, W3URL
from urllib.parse import urlencode

In [68]:
code='''
from pyr.app.k12ai import EasyaiClassifier, EasyaiTrainer, EasyaiDataset
from collections import OrderedDict
import torch
from torch import nn

class CustomClassifier(EasyaiClassifier):
    ## Load
    def prepare_dataset(self):
        return self.load_presetting_dataset_('rmnist', '/data/datasets/cv')
    
    def build_model(self):
        # return self.load_pretrained_model_('resnet18', num_classes=10)
        class SmallCNN(nn.Module):
            def __init__(self, num_classes=10, hidden_size=100):
                super(SmallCNN, self).__init__()
                self.features = nn.Sequential(
                    nn.Conv2d(3, 32, 3, padding=1),             # 128: (128-3+2*1)//1 + 1
                    nn.ReLU(inplace=True),
                    nn.Conv2d(32, 32, 3, padding=1, stride =1), # 128: (128-3+2*1)//1 + 1
                    nn.ReLU(inplace=True),
                    nn.Conv2d(32, 32, 3, padding=1, stride =1), # 128: (128-3+2*1)//1 + 1
                    nn.ReLU(inplace=True),
                    nn.Conv2d(32, 32, 3, padding=1, stride=2),  # 64: (128-3+2*1)//2 + 1
                    nn.ReLU(inplace=True),
                    nn.Conv2d(32, 64, 3, padding=1),            # 64: (128-3+2*1)//1 + 1  
                    nn.ReLU(inplace=True),
                    nn.Conv2d(64, 64, 3, padding=1, stride=2),  # 32: (64-3+2*1)//2 + 1
                    nn.ReLU(inplace=True))
                self.classifier = nn.Sequential(
                    nn.Flatten(),                               # 64*32*32
                    nn.Linear(64*32*32, hidden_size), nn.ReLU(),
                    nn.Linear(hidden_size, num_classes))

            def forward(self, x):
                x = self.features(x)
                x = self.classifier(x)
                return x
        return SmallCNN(num_classes=10)
    
    def configure_optimizer(self, model):
        return self.adam(model.parameters(),
            base_lr=0.001)

    def configure_scheduler(self, optimizer):
        return self.period_step(optimizer, step_size=30, gamma=0.1)
    
    ## Train
    def train_dataloader(self):
        return self.get_dataloader(
            phase='train',
            data_augment=[
                self.random_resized_crop(size=(128, 128)),
                self.random_brightness(factor=0.3),
                self.random_rotation(degrees=30)
            ],
            random_order=False,
            input_size=128,
            normalize=True,
            batch_size=32,
            drop_last=False,
            shuffle=False)

    def training_step(self, batch, batch_idx):
        # REQUIRED
        x, y, _ = batch
        y_hat = self.forward(x) # (32, 10)
        loss = self.cross_entropy(y_hat, y, reduction='mean')
        with torch.no_grad():
            accuracy = (torch.argmax(y_hat, axis=1) == y).float().mean()
        log = {'train_loss': loss, 'train_acc': accuracy}
        output = OrderedDict({
            'loss': loss,        # M
            'acc': accuracy,     # O
            'progress_bar': log, # O
        })
        return output

    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['acc'] for x in outputs]).mean()
        log = {'train_loss': avg_loss, 'train_acc': avg_acc}
        output = OrderedDict({
            'progress_loss': log,
        })
        return output
        
    ## Valid
    def val_dataloader(self):
        return self.get_dataloader('val',
                input_size=128,
                batch_size=32,
                drop_last=False,
                shuffle=False)

    def validation_step(self, batch, batch_idx):
        x, y, _ = batch
        y_hat = self.forward(x) # (32, 10)
        loss = self.cross_entropy(y_hat, y, reduction='mean')
        accuracy = (torch.argmax(y_hat, axis=1) == y).float().mean()
        output = OrderedDict({
            'val_loss': loss,
            'val_acc': accuracy,
        })
        return output

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
        log = {'val_loss': avg_loss, 'val_acc': avg_acc}
        output = OrderedDict({
            **log,
            'progress_loss': log,
        })
        return output
        
    ## Test
    def test_dataloader(self):
        return self.get_dataloader('test',
                input_size=128,
                batch_size=32,
                drop_last=False,
                shuffle=False)

    def test_step(self, batch, batch_idx):
        x, y, _ = batch
        y_hat = self.forward(x) # (32, 10)
        loss = self.cross_entropy(y_hat, y, reduction='mean')
        accuracy = (torch.argmax(y_hat, axis=1) == y).float().mean()
        output = OrderedDict({
            'test_loss': loss,
            'test_acc': accuracy
        })
        return output

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['test_acc'] for x in outputs]).mean()
        log = {'test_loss': avg_loss, 'test_acc': avg_acc}
        output = OrderedDict({
            'progress_bar': log,
            **log
        })
        return output

        
trainer = EasyaiTrainer(max_epochs=1, model_summary=None, early_stop={'monitor': 'val_acc', 'patience': 3, 'mode': 'max'})
trainer.fit(CustomClassifier())
print(trainer.test())
'''

In [66]:
exec(code)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


--------------------------------------------------------------------------------
TEST RESULTS
{'test_acc': tensor(0.1052, device='cuda:0'),
 'test_loss': tensor(2.5602, device='cuda:0')}
--------------------------------------------------------------------------------

{'test_loss': 2.560201406478882, 'test_acc': 0.1051587387919426}


In [69]:
params = {'default': code.replace('pyr.app.', '').replace('/data/datasets/cv', '/datasets')}
k12ai_start_html(f'{W3URL}/codemirror.html?{urlencode(params)}', width='100%', height='900px')